In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [1]:
!pip install git+https://github.com/keras-team/keras-preprocessing.git

  Cloning https://github.com/keras-team/keras-preprocessing.git to c:\users\batcomputer\appdata\local\temp\pip-req-build-tary93ud
  Resolved https://github.com/keras-team/keras-preprocessing.git to commit 3e380065d4afc7347aaee8d89325a16b22158438
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-preprocessing.git 'C:\Users\BATCOMPUTER\AppData\Local\Temp\pip-req-build-tary93ud'


In [2]:
from keras_preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
traindf=pd.read_csv("C:/Users/BATCOMPUTER/Desktop/DHPi_competition_data_sprint_68/Dataset/training_set.csv",dtype=str)
testdf=pd.read_csv("C:/Users/BATCOMPUTER/Desktop/DHPi_competition_data_sprint_68/Dataset/Testing_set.csv",dtype=str)

In [6]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/BATCOMPUTER/Desktop/DHPi_competition_data_sprint_68/Dataset/training_dataset",
x_col="filename",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 233 validated image filenames belonging to 16 classes.


In [8]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/BATCOMPUTER/Desktop/DHPi_competition_data_sprint_68/Dataset/training_dataset",
x_col="filename",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 77 validated image filenames belonging to 16 classes.


In [9]:
test_datagen=ImageDataGenerator(rescale=1./255.)

In [10]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="C:/Users/BATCOMPUTER/Desktop/DHPi_competition_data_sprint_68/Dataset/test",
x_col="filename",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

Found 13950 validated image filenames.


In [11]:
IMAGE_SIZE = [224, 224]
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [12]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [13]:
x = Flatten()(resnet.output)

In [14]:
prediction = Dense(16, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [17]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50
)

C:\Users\BATCOM~1\AppData\Local\Temp/ipykernel_16880/2070787754.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/50
7/7 [==============================] - 9s 1s/step - loss: 10.2063 - accuracy: 0.1095 - val_loss: 58.9834 - val_accuracy: 0.0000e+00
Epoch 2/50
7/7 [==============================] - 6s 890ms/step - loss: 9.7185 - accuracy: 0.1841 - val_loss: 81.6472 - val_accuracy: 0.0000e+00
Epoch 3/50
7/7 [==============================] - 6s 982ms/step - loss: 8.7223 - accuracy: 0.1841 - val_loss: 80.9148 - val_accuracy: 0.0000e+00
Epoch 4/50
7/7 [==============================] - 6s 892ms/step - loss: 5.0770 - accuracy: 0.2587 - val_loss: 78.7083 - val_accuracy: 0.0000e+00
Epoch 5/50
7/7 [==============================] - 6s 915ms/step - loss: 3.5228 - accuracy: 0.3881 - val_loss: 78.7399 - val_accuracy: 0.0000e+00
Epoch 6/50
7/7 [==============================] - 6s 893ms/step - loss: 2.6936 - accuracy: 0.4378 - val_loss: 76.5134 - val_accuracy: 0.0000e+00
Epoch 7/50
7/7 [==============================] - 6s 891ms/step - loss: 1.9624 - accuracy: 0.4478 - val_loss: 76.4819 - val_accuracy

In [19]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST)

C:\Users\BATCOM~1\AppData\Local\Temp/ipykernel_16880/2180879535.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=valid_generator,


[78.1743392944336, 0.10389610379934311]

In [20]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

C:\Users\BATCOM~1\AppData\Local\Temp/ipykernel_16880/2076935646.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator,


435/435 [==============================] - 321s 737ms/step


In [22]:
from tensorflow.keras.models import load_model

model.save('model_resnet_pest.h5')

In [23]:
predicted_class_indices=np.argmax(pred,axis=1)

In [24]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [29]:
print(len(predictions))

13920


In [30]:
import statistics as st

filenames=test_generator.filenames

# if length are not equal
if len(filenames) != len(predictions):
    # Append mean values to the list with smaller length
    if len(filenames) > len(predictions):
        mode_predictions = st.mode(predictions)
        predictions += (len(filenames)-len(predictions)) * [mode_predictions]
        
results=pd.DataFrame({"filename":filenames,
                      "label":predictions})
results.to_csv("results.csv",index=False)